In [1]:
!apt-get install openjdk-8-jdk-headless -qq 
!wget -q !wget -q https://downloads.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz  
!tar -xf spark-3.3.2-bin-hadoop3.tgz
!pip install -q findspark

import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

findspark.init()
findspark.find()

Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 128276 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u362-ga-0ubuntu1~20.04.1_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u362-ga-0ubuntu1~20.04.1) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.
Preparing to unpack .../openjdk-8-jdk-headless_8u362-ga-0ubuntu1~20.04.1_amd64.deb ...
Unpacking openjdk-8-jdk-headless:amd64 (8u362-ga-0ubuntu1~20.04.1) ...
Setting up openjdk-8-jre-headless:amd64 (8u362-ga-0ubuntu1~20.04.1) ...
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/orbd to provide /usr/bin/orbd (orbd) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/servertool to provide /usr/bin/servertool (servertool) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/tnameserv to provide /usr/bin/tnameserv (tnameserv) in auto mode
Setting u

'/content/spark-3.3.2-bin-hadoop3'

# Logistic Regression

- 참고 : https://spark.apache.org/docs/latest/ml-classification-regression.html

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('mylogreg').getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

## Load training data

In [4]:
!ls /content/spark-3.3.2-bin-hadoop3/sample_libsvm_data.txt

ls: cannot access '/content/spark-3.3.2-bin-hadoop3/sample_libsvm_data.txt': No such file or directory


In [5]:
my_data = spark.read.format("libsvm").load("/content/spark-3.3.2-bin-hadoop3/data/mllib/sample_libsvm_data.txt")

In [6]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



## Fit the model

In [7]:
model = LogisticRegression()

In [8]:
fitted_model = model.fit(my_data)

In [9]:
log_summary = fitted_model.summary

In [10]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [11]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000486...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716171...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298141...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113070...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504196...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212814...|[6.97903542823781...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446123...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606582...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

## train_data, test_data 데이터셋 분리

In [12]:
train, test = my_data.randomSplit([0.7, 0.3])

## Fit the model

In [13]:
final_model = LogisticRegression()

In [14]:
fit_final = final_model.fit(train)

## Evaluation

In [15]:
prediction_and_labels = fit_final.evaluate(train)

In [16]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[98,99,100,1...|[34.2318611920029...|[0.99999999999999...|       0.0|
|  0.0|(692,[100,101,102...|[19.6299291076469...|[0.99999999701578...|       0.0|
|  0.0|(692,[121,122,123...|[24.6090784885047...|[0.99999999997946...|       0.0|
|  0.0|(692,[122,123,148...|[21.1120287520953...|[0.99999999932210...|       0.0|
|  0.0|(692,[123,124,125...|[36.4273701425704...|[0.99999999999999...|       0.0|
|  0.0|(692,[123,124,125...|[23.2372852707688...|[0.99999999991905...|       0.0|
|  0.0|(692,[124,125,126...|[32.3698255807588...|[0.99999999999999...|       0.0|
|  0.0|(692,[124,125,126...|[22.6031601309448...|[0.99999999984739...|       0.0|
|  0.0|(692,[124,125,126...|[28.0439552620378...|[0.99999999999933...|       0.0|
|  0.0|(692,[124

In [17]:
from pyspark.ml.evaluation import ( BinaryClassificationEvaluator, 
                                 MulticlassClassificationEvaluator)

In [18]:
my_eval = BinaryClassificationEvaluator()

In [19]:
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)

In [20]:
my_final_roc

1.0